In [3]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

api_key = "dnDSPYh1mNwZyGRSQgjCPKQc144MfMng"
model = "mistral-large-latest"

client = MistralClient(api_key=api_key)

In [ ]:
chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="user", content="What is the best French cheese?")]
)

print(chat_response.choices[0].message.content)

In [64]:
def get_mistral_compare_items(item_A, item_B):
    """
    Asks Mistral to compare two items from the list and say if it's the same thing 
    """
    #step 1
    chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="system", content='You will be asked questions about items to determine if they are the same name for a product inside grocery store. Specifically, you will need to assess if item_A and item_B are the same in terms of being found in the same section or location within the store. For example, "картошка" and "картофелина" should be considered the same since they can be bought in the same place, and you should answer "yes". Please answer with "yes" or "no"'),
              ChatMessage(role="user", content=f'If "{item_A}" and "{item_B}" are the same?')]
    )
    res1 = chat_response.choices[0].message.content
    
    # step 2
    chat_response2 = client.chat(
    model=model,
    messages=[ChatMessage(role="system", content='You will be given an answer to a certain question and you are need to summarize the answer as "yes" or "no". Answer only with "yes" or "no".'),
              ChatMessage(role="user", content=f'Does this text means "yes" or "no"? text:{res1}')]
    )
    res2 = chat_response2.choices[0].message.content
 
    if "yes" in res2.lower(): #need to properly regex this
        return True
    elif "no" in res2.lower():
        return False
    else:
        raise Exception("Mistral was unable to answer 'yes' or 'no'") 


In [110]:
def get_mistral_convert_units(item):
    """
    Asks Mistral to compare two items from the list and say if it's the same thing 
    """
    #step 1
    chat_response = client.chat(
    model=model, #
    messages=[ChatMessage(role="user", content=f'Please convert this unit into the International System of Units: "1 bottle of vodka" I need it to go to shop in France and I want all items to be represented in the same measurements. I want pack of milk represented as 1 liter, 1 potato as 0.15 kg and so on. Please provide short answer')])
    res1 = chat_response.choices[0].message.content
    
        # step 2
    chat_response2 = client.chat(
    model=model,
    messages=[ChatMessage(role="system", content='Extract number and SI unit from the given text. Format your text like "10 kilograms" or "1 litre"'),
              ChatMessage(role="user", content=f'text:{res1}')]
    )
    res2 = chat_response2.choices[0].message.content
    
            # step 3
    chat_response3 = client.chat(
    model=model,
    messages=[ChatMessage(role="system", content='Extract number and SI unit from the given text. Format your text like "10 kilograms" or "1 litre". Answer only with number and unit, nothing else.'),
              ChatMessage(role="user", content=f'text:{res2}')]
    )
    res3 = chat_response3.choices[0].message.content
    
    return res3


    


In [111]:
get_mistral_convert_units("s")

'0.75 liters'

In [17]:
get_mistral_answer('From the point of view of finding these items in a grocery store, are "бульба" and "potatoes" the same? Please answer with "yes" or "no" only, with no further explanations.')

'No. "Бульба" is a Ukrainian word that can refer to various root vegetables, not just potatoes. In English, it\'s often translated as "tuber" or "root crop". While it can include potatoes, it may also refer to beets, carrots, or other root vegetables. So, in a grocery store, "бульба" and "potatoes" would not necessarily refer to the same thing.'

In [21]:
get_mistral_answer("""Does this text means "yes" or "no"? Please answer with "yes" or "no" only. 
                   The text: No. "Бульба" is a Ukrainian word that can refer to various root vegetables, not just potatoes. In English, it\'s often translated as "tuber" or "root crop". While it can include potatoes, it may also refer to beets, carrots, or other root vegetables. So, in a grocery store, "бульба" and "potatoes" would not necessarily refer to the same thing""")

'No.'

In [112]:
import pandas as pd

In [173]:
data_df = pd.DataFrame(
    {
        "To buy": ["potatoes", "eggs", "eggs", "яйца"],
        "Amount": [3, 12, 2, 1],
        "Unit": ["kg", "pieces", "counts", "packs"],
        "Done": [False, False, False, False]
    }
)

In [174]:
data_df

,To buy,Amount,Unit,Done
0,potatoes,3,kg,False
1,eggs,12,pieces,False
2,eggs,2,counts,False
3,яйца,1,packs,False


In [177]:
def filter_shopping_list(data_df):
    for i in data_df.index:
        for j in data_df.index:
            if i != j and data_df.loc[i, 'To buy'] is not None and data_df.loc[j, 'To buy'] is not None:
                if data_df.loc[i, 'To buy'].lower() == data_df.loc[j, 'To buy'].lower():
                    data_df.loc[i,'Amount'] += data_df.loc[j,'Amount']
                    data_df.loc[j, 'To buy'] = None
                else:
                    try:
                        if get_mistral_compare_items(data_df.loc[i, 'To buy'], data_df.loc[j, 'To buy']) == True:
                            data_df.loc[i,'Amount'] += data_df.loc[j,'Amount']
                            data_df.loc[j, 'To buy'] = None
                    except Exception as e:
                        print(e)
    data_df = data_df.dropna()
    return data_df

In [178]:
filter_shopping_list(data_df)

No, "potatoes" and "eggs" are not the same and are usually located in different sections of a grocery store. Potatoes are typically found in the produce section, while eggs are usually located in the dairy or refrigerated section.
No.
No, "potatoes" and "eggs" are not the same, as they are typically found in different sections of a grocery store. Potatoes are usually found in the produce section, while eggs are typically found in the dairy or refrigerated section.
No.
No, "potatoes" and "яйца" (which means "eggs" in English) are not the same. They are found in different sections of a grocery store. Potatoes are typically found in the produce section, while eggs are usually found in the dairy or refrigerated section.
No.
No, eggs and potatoes are not the same in terms of being found in the same section or location within the store. Eggs are typically found in the dairy section, while potatoes are found in the produce section.
No.
Yes, "eggs" and "яйца" are the same, as they refer to the

,To buy,Amount,Unit,Done
0,potatoes,3,kg,False
1,eggs,15,pieces,False


In [184]:
def mistral_create_csv(input):
    """
    Asks Mistral to create csv from the text of the list
    """
    #step 1
    chat_response = client.chat(
    model=model, #
    messages=[ChatMessage(role="user", content=f"""Organize this shopping list in a csv file with columns: Item, Amount, Unit. Write it so I can open it in pandas later, so no additional text or comments.
                          {input}""")])
    res1 = chat_response.choices[0].message.content
    return res1
    

In [192]:
string_data = StringIO(mistral_create_csv("""✓ рыба
    ✓ листы для шаурмы или листья салата
    ✓ сухарики
    ✓ фрукты для салата и на десерт 
    ✓ авокадо для завтрака
    ✓ 350 g d' oignon
    ✓ авокадо 
    ✓ огурец
    ✓ 350 g de poivron de couleur rouge et vert
    ✓ 350 g de courgette
    ✓ 500 g de tomate bien mûres"""))

In [193]:
df = pd.read_csv(string_data)
df

,Item,Amount,Unit
0,рыба,NaN,NaN
1,листы для шаурмы или листья салата,NaN,NaN
2,сухарики,NaN,NaN
3,фрукты для салата и на десерт,NaN,NaN
4,авокадо для завтрака,NaN,NaN
5,авокадо,NaN,NaN
6,огурец,NaN,NaN
7,oignon,350.0,g
8,poivron de couleur rouge et vert,350.0,g
9,courgette,350.0,g


In [ ]:
def create_csv(input):
    string_data = StringIO(mistral_create_csv(f"{input}"))
    return pd.read_csv(string_data)
    

In [221]:
input_list = []
for i in df.index:
    for j in df.index:
        input_list.append((df.loc[i, 'Item'], df.loc[j, 'Item']))
        

In [222]:
texts = []
for pair in input_list:
    texts.append(f'Are "{pair[0]}" and "{pair[1]}" the same?')

    

In [223]:
import concurrent

In [224]:
#step 1
def mistral_compare(text):
    chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="system", content='You will be asked questions about items to determine if they are the same name for a product inside grocery store. Specifically, you will need to assess if item_A and item_B are the same in terms of being found in the same section or location within the store. For example, "картошка" and "картофелина" should be considered the same since they can be bought in the same place, and you should answer "yes". Please answer with "yes" or "no"'),
                ChatMessage(role="user", content=f'{text}')]
    )
    res1 = chat_response.choices[0].message.content
    return res1


In [226]:
text_results = list(results)

In [231]:
text_results[0].lower().startswith('yes')

True

In [233]:
def mistral_compare_items(data_df=df):
    input_list = []
    for i in data_df.index:
        for j in data_df.index:
            input_list.append((data_df.loc[i, 'Item'], data_df.loc[j, 'Item']))
            
    texts = []
    for pair in input_list:
        texts.append(f'Are "{pair[0]}" and "{pair[1]}" the same?')
        
    with concurrent.futures.ThreadPoolExecutor(max_workers=len(texts)) as pool:
        results = pool.map(mistral_compare, texts)
    text_results = list(results)    
            
            
    for i in data_df.index:
        for j in data_df.index:
            if i != j and data_df.loc[i, 'Item'] is not None and data_df.loc[j, 'Item'] is not None:
                if data_df.loc[i, 'Item'].lower() == data_df.loc[j, 'Item'].lower():
                    data_df.loc[i,'Amount'] += data_df.loc[j,'Amount']
                    data_df.loc[j, 'Item'] = None
                else:            
                    if text_results[i*len(data_df) + j].lower().startswith('yes') == True:
                        data_df.loc[i,'Amount'] += data_df.loc[j,'Amount']
                        data_df.loc[j, 'Item'] = None
                    elif text_results[i*len(data_df) + j].lower().startswith('no') == True:
                        pass
                    else:
                        raise Exception("Mistral was unable to answer 'yes' or 'no'")
    return data_df

In [234]:
mistral_compare_items()

,Item,Amount,Unit
0,рыба,NaN,NaN
1,листы для шаурмы или листья салата,NaN,NaN
2,сухарики,NaN,NaN
3,фрукты для салата и на десерт,NaN,NaN
4,авокадо для завтрака,NaN,NaN
5,None,NaN,NaN
6,None,NaN,NaN
7,oignon,350.0,g
8,poivron de couleur rouge et vert,350.0,g
9,courgette,350.0,g


In [194]:
def get_mistral_compare_items(item_A, item_B):
    """
    Asks Mistral to compare two items from the list and say if it's the same thing 
    """
    
    
    #step 1
    chat_response = client.chat(
    model=model,
    messages=[ChatMessage(role="system", content='You will be asked questions about items to determine if they are the same name for a product inside grocery store. Specifically, you will need to assess if item_A and item_B are the same in terms of being found in the same section or location within the store. For example, "картошка" and "картофелина" should be considered the same since they can be bought in the same place, and you should answer "yes". Please answer with "yes" or "no"'),
              ChatMessage(role="user", content=f'Are "{item_A}" and "{item_B}" the same?')]
    )
    res1 = chat_response.choices[0].message.content
    
    if "yes" in res1.lower(): #need to properly regex this
        return True
    elif "no" in res1.lower():
        return False
    else:
        #raise Exception("Mistral was unable to answer 'yes' or 'no'")
            # step 2
        chat_response2 = client.chat(
        model=model,
        messages=[ChatMessage(role="system", content='You will be given an answer to a certain question and you are need to summarize the answer as "yes" or "no". Answer only with "yes" or "no".'),
                ChatMessage(role="user", content=f'Does this text means "yes" or "no"? text:{res1}')]
        )
        res2 = chat_response2.choices[0].message.content
        
        if "yes" in res2.lower(): #need to properly regex this
            return True
        elif "no" in res2.lower():
            return False
        else:
            raise Exception("Mistral was unable to answer 'yes' or 'no'")

In [195]:
get_mistral_compare_items('egg', 'oeuf')

True